In [0]:
""" Sudoku board Class and solver logic. """
from collections import Counter
from copy import deepcopy
from math import sqrt
from typing import List, Tuple
import numpy as np


def create_mask(board: np.ndarray, dimensions: Tuple[int, int], box_size: int) -> List[List[int]]:
    """ Function to create Mask of possible valid values based on the initial sudoku Board. """

    mask = list(board.tolist())
    counts = Counter(board.flatten())
    del counts[0]
    counts = [number[0] for number in counts.most_common()]
    most_common_clues = counts
    for clue in range(dimensions[0], dimensions[1]):
        if clue not in most_common_clues:
            most_common_clues.append(clue)

    for i, row in enumerate(mask):
        if 0 in row:
            while 0 in row:
                zero_index = row.index(0)
                mask[i][zero_index] = []
                for number in most_common_clues:
                    if valid(board, number, (i, zero_index), box_size):
                        mask[i][zero_index].append(number)
        else:
            for number in row:
                if number != 0:
                    mask[i][row.index(number)] = {number}
    return mask


def update_mask(board: np.ndarray, mask: List[List[int]], box_size: int) -> List[List[int]]:
    """ Function to update Mask of possible valid values. """

    def is_list(item):
        return bool(isinstance(item, list))

    for y_pos, row in enumerate(mask):
        for numbers in filter(is_list, row):
            x_pos = row.index(numbers)
            to_remove = set()
            for number in numbers:
                if not valid(board, number, (y_pos, x_pos), box_size):
                    to_remove.add(number)
            for num in to_remove:
                mask[y_pos][x_pos].remove(num)
    return mask


def update_board(board: np.ndarray, mask: List[List[int]]) -> (np.ndarray, [List[int]]):
    """ Function to update Board based on possible values Mask. """

    def is_one_element_list(item):
        return bool(isinstance(item, list) and len(item) == 1)

    for y_pos, row in enumerate(mask):
        for number in filter(is_one_element_list, row):
            x_pos = row.index(number)
            num = number.pop()
            board[y_pos][x_pos] = num
    return board, mask


def preprocess_board(board: np.ndarray, box_size: int, dimensions: (int, int)) -> (np.ndarray, [List[int]]):
    """ Board preprocessor to reduce necessary iterations during solving. """

    mask = create_mask(board, dimensions, box_size)
    temp_mask = None
    passes = 0

    while temp_mask != mask:
        passes += 1
        temp_mask = deepcopy(mask)
        mask = update_mask(board, mask, box_size)
        board, mask = update_board(board, mask)

    #if args.verbose:
    #    print(f'Preprocess passes: {passes}')

    return np.array(board), mask


def solve(board: np.ndarray, mask, size: int, box_size: int, dimensions: Tuple[int, int]) -> bool:
    """ Function to solve Sudoku with backtracking. """
    solve.iterations += 1

    find = find_min_empty(board, mask)
    # find = find_empty(board)  # Old method for finding empty cells.
    if not find:
        return True
    row, col = find

    for number in mask[row][col]:
        if valid(board, number, (row, col), box_size):
            board[row, col] = number
            if solve(board, mask, size, box_size, dimensions):
                return True
            board[row, col] = 0

    return False


def valid(board: np.ndarray, number: int, pos: Tuple[int, int], box_size: int) -> bool:
    """ Function to check if a given value is valid for the specific location of the sudoku. """
    # Check row
    location = np.where(board[pos[0]] == number)
    if len(location[0]):
        return False

    # Check column
    location = np.where(board[:, pos[1]] == number)
    if len(location[0]):
        return False

    # Check box
    box_x = pos[1] // box_size
    box_y = pos[0] // box_size
    location = np.where(board[(box_y * box_size):(box_y * box_size + box_size),
                   (box_x * box_size):(box_x * box_size + box_size)] == number)

    if len(location[0]):
        return False
    return True


def print_board(board: np.ndarray, size: int, box_size: int) -> None:
    """ Pretty print Sudoku board. """
    for i, row in enumerate(board):
        if i % box_size == 0 and i != 0:
            print('- - - - - - - - - - - -')

        for j, _ in enumerate(row):
            if j % box_size == 0 and j != 0:
                print(' | ', end='')

            if j == (size - 1):
                print(board[i, j])
            else:
                print(str(board[i, j]) + ' ', end='')


def print_mask(mask_to_print: List[List[int]], box_size: int) -> None:
    """ Pretty print Mask of possible valid values. """
    for i, row in enumerate(mask_to_print):
        if i % box_size == 0 and i != 0:
            print('- - - - - - - - - - - -')
        print(row)


def find_empty(board: np.ndarray) -> Tuple[int, int] or None:
    """ Find empty location to be filled in Sudoku. """
    location = np.argwhere(board == 0)
    if np.any(location):
        return location[0][0], location[0][1]  # row, column

    return None


def find_min_empty(board: np.ndarray, mask: List[List[int]]) -> Tuple[int, int] or None:
    """ Find empty location to be filled in Sudoku,
        where the number of possible values is optimal. """

    def not_zero_element_list(item):
        return bool(isinstance(item, list) and len(item) != 0)

    shortest_cue_lists = {}
    for y_pos, row in enumerate(mask):
        sorted_lists = sorted(filter(not_zero_element_list, row), key=len, reverse=True)
        if len(sorted_lists) >= 1:
            for item in sorted_lists:
                shortest_cue_lists[(y_pos, row.index(item))] = len(sorted_lists[0])
    shortest_cue_lists = dict(sorted(shortest_cue_lists.items(), key=lambda item: item[1]))

    for coordinate in shortest_cue_lists.keys():
        if board[coordinate[0]][coordinate[1]] == 0:
            return coordinate[0], coordinate[1]  # row, column

    return find_empty(board)


In [0]:
def parse_sudoku_string(data):
    """ Helper function to parse sudoku challenge. """
    size = int(sqrt(len(data)))
    box_size = int(sqrt(size))
    dimensions = (1, size + 1)
    board = []
    row = []
    for i, item in enumerate(data):
        row.append(int(item))
        if (i + 1) % size == 0:
            board.append(row)
            row = []
    return np.array(board)

In [0]:
challenge = parse_sudoku_string("900000000060000000027008000000000307890300000301020580000100800080075602010600009")
print(challenge)
print(type(challenge))
print_board(challenge, 9, 3)

[[9 0 0 0 0 0 0 0 0]
 [0 6 0 0 0 0 0 0 0]
 [0 2 7 0 0 8 0 0 0]
 [0 0 0 0 0 0 3 0 7]
 [8 9 0 3 0 0 0 0 0]
 [3 0 1 0 2 0 5 8 0]
 [0 0 0 1 0 0 8 0 0]
 [0 8 0 0 7 5 6 0 2]
 [0 1 0 6 0 0 0 0 9]]
<class 'numpy.ndarray'>
9 0 0  | 0 0 0  | 0 0 0
0 6 0  | 0 0 0  | 0 0 0
0 2 7  | 0 0 8  | 0 0 0
- - - - - - - - - - - -
0 0 0  | 0 0 0  | 3 0 7
8 9 0  | 3 0 0  | 0 0 0
3 0 1  | 0 2 0  | 5 8 0
- - - - - - - - - - - -
0 0 0  | 1 0 0  | 8 0 0
0 8 0  | 0 7 5  | 6 0 2
0 1 0  | 6 0 0  | 0 0 9


In [0]:
from pyspark import SparkFiles

sc = spark.sparkContext
sc.addFile("https://raw.githubusercontent.com/kasztp/kiwi.com-sudoku-solver/master/sudokus/20_hard_sudokus.txt")
sc.addFile("https://raw.githubusercontent.com/kasztp/kiwi.com-sudoku-solver/master/sudokus/100_hard_sudokus.txt")
sc.addFile("https://raw.githubusercontent.com/kasztp/kiwi.com-sudoku-solver/master/sudokus/1000_hard_sudokus.txt")
sc.addFile("https://raw.githubusercontent.com/kasztp/kiwi.com-sudoku-solver/master/sudokus/10k_hard_sudokus.txt")

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


def solver(data):
    solve.iterations = 0
    challenge, size, box_size, dimensions = parse_sudoku_string(data), 9, 3, (1, 10)
    sudoku, mask = preprocess_board(challenge, box_size, dimensions)
    solve(sudoku, mask, size, box_size, dimensions)
    solution = ''
    for row in sudoku:
        for number in row:
            solution += str(number)
    return solution

udf_solver = udf(solver, StringType())

In [0]:
sudoku_df = (
    spark.read.text("file://"+SparkFiles.get("20_hard_sudokus.txt"))
)

sudoku_df = sudoku_df.filter(sudoku_df['value'] != '20').withColumnRenamed('value', 'Original')

In [0]:
sudoku_df.show(20, False)

+---------------------------------------------------------------------------------+
|Original                                                                         |
+---------------------------------------------------------------------------------+
|000075400000000008080190000300001060000000034000068170204000603900000020530200000|
|300000000050703008000028070700000043000000000003904105400300800100040000968000200|
|302609005500730000000000900000940000000000109000057060008500006000000003019082040|
|530000008007000030200006901000500200090370004000981000300040560000090000000007080|
|008310900095000160000000005000400000000080049006072000000001030000240607001008200|
|000400970000051600042000010030000000070508064000070000700030000300090000005864009|
|060500000720000000000000320000050637000004500000230180180009000603070000004006003|
|274000030000000005000600041900306000100280000006054000000000002007000583000095700|
|570000069000003800090000000801600000000030600702000050000060501000702000006

In [0]:
print(f'Before: {sudoku_df.rdd.getNumPartitions()}')
sudoku_df = sudoku_df.repartition(20).cache()
print(f'After: {sudoku_df.rdd.getNumPartitions()}')

Before: 1
After: 20


In [0]:
solved_df = sudoku_df.withColumn('Solution', udf_solver(sudoku_df['Original']))

In [0]:
solved_df.show(20, False)

+---------------------------------------------------------------------------------+---------------------------------------------------------------------------------+
|Original                                                                         |Solution                                                                         |
+---------------------------------------------------------------------------------+---------------------------------------------------------------------------------+
|400500600200000000000020000002004380000030000790000504000060490070093810500100030|438519672216478953957326148162754389845932761793681524321865497674293815589147236|
|005200000400300700600000010800020100040800500000095000083040070090006080500902000|735218964418369752629457318857624193941873526362195847283541679194736285576982431|
|130400000705300000600020000000000027000900400000000085860500003059103000002004060|138479652725316894694825731541638927283957416976241385867592143459163278312784569|
|570

In [0]:
print(solved_df.rdd.getNumPartitions())

20
